# Chapter 09: -2- setup.py and CLI.py for the entry point
2020-11-16

Content:
1. UML diagram of the unitconver package
2. **setup.py and CLI.py for the entry point (and argParse module)**
3. UnitTable.py and the unit_tables directory (and lambda function)
4. Converter.py (and importlib module)
5. Putting all of them together!

## 幾個重要的概念：

1. Chapter 05 用 os.argv[**]
2. Chapter 09 用 argparse 的方法：提供 positional/optional argument，之後作相對應的處理，其中並包括預測的 -h 等 help 的選項

In [1]:
import os
course_directory = 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release'
os.chdir(course_directory + "/Chapter09/unitconverter")

In [ ]:
"""
setup.py for unitconverter
這不可以執行，但我列出來，方便參考。
"""
from setuptools import setup

setup(
    name='unitconverter',
    version='0.1.0',
    entry_points={
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
    },
    description='Command line tool for unit conversion',
    classifiers=[
        'Natural Language :: English',
        'Programming Language :: Python :: 3.5',
        'Programming Language :: Python :: 3.6',
        'Programming Language :: Python :: 3.7',
    ],
    author='Dan Nixon',
    packages=['unitconverter', 'unitconverter.unit_tables'],
    include_package_data=True,
    zip_safe=False)

In [2]:
!unitconvert -h

usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


## 相關關鍵字介紹：
- positional arguments
- optional arguments (就是有用 --全稱，或是 -簡寫 標註的參數)
- parser
- subparser

### For example: unitconvert time list -m
- unitconvert: entry point
- time: parser argument
- list: one of the subparsers (covert is the other one in this case)
- -m: optional argument of the designated subparser

## 撰寫 unitconverter 的 Command-line Interface 程式

Command-line Interface(CLI) 的設定全部都包含在 CLI.py 中，\
CLI 指令供分為兩層

1. 第一層 (parser) 用於選擇所需要的unit_table的類型
2. *第二層 (subparser) 用於選擇*：
    + {列出 table 中單位的 list 模式} 或 
        - list 模式用於顯示選定的 unit_table 可以轉換的所有單位的名稱
    + {進入使用 convert 的模式}
        - conversion 模式用於單位的轉換

In [ ]:
'''
CLI.py 內容：  
'''

import argparse
import inspect
from unitconverter.Converter import get_table, convert_units


def run_cli():
    parser = argparse.ArgumentParser(description='Tool for converting units')

    parser.add_argument(
        'table',
        metavar='TABLE',
        action='store',
        type=str,
        help='Unit table to use in conversion'
    )

    """
    這是 unitconvert 以及上述 table 之後，第二層要解碼的部份，
    這又分為兩個：
    1. list
    2. convert
    我們安排一個 which 變數來記錄是哪一個。
    """
    subparsers = parser.add_subparsers(help='operation to be performed')

    '''
    以下考慮 -m, --method 的選項，要作什麼事
    
    Unit table energy can convert between the units:
    kcal (x * 0.00023884589663)
    cal (x * 0.23900573614)
    btu (x * 0.00094781707775)
    ev (x * 6241506480000000000.0)
    wh (x * 0.00027777777778)
    j (base unit)
    hph (x * 3.7250614123e-7)
    '''
    
    list_table_parser = subparsers.add_parser('list')
    list_table_parser.set_defaults(which='list')

    list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )
    
    '''
    以下考慮 convert 時，要作什麼事
    
    usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

    positional arguments:
      VALUE       The value to convert
      FROM        Unit to convert from
      TO          Unit(s) to convert to

    optional arguments:
      -h, --help  show this help message and exit
    '''

    conversion_parser = subparsers.add_parser('convert')
    conversion_parser.set_defaults(which='convert')

    conversion_parser.add_argument(
        'value',
        metavar='VALUE',
        type=float,
        action='store',
        help='The value to convert'
    )

    conversion_parser.add_argument(
        'from_unit',
        metavar='FROM',
        action='store',
        type=str,
        help='Unit to convert from'
    )

    conversion_parser.add_argument(
        'to_units',
        metavar='TO',
        action='store',
        nargs='+', # 個數可以有很多個，但至少會有一個
        type=str,
        help='Unit(s) to convert to'
    )

    """
    以下這個指令 parser.parse_args() 是最重要的，
    將 CLI 輸入的參數按指定的名稱取出，
    然後，就可以作判斷，要作怎樣的反應。
    """
    props = parser.parse_args()

    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)


def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:' % props.table)
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                """
                self.from_base_unit['radian'] = lambda x: x * 0.0174532925
                """
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip() # 這是要將一開頭的 [lambda x:] 給去掉
            print('%s (%s)' % (unit, formula))
        else:
            print(unit)


def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print('%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit']))

In [5]:
'''
比較常看到的方式 -h 或是 --help
'''
!unitconvert -h

usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [6]:
!unitconvert TABLE

Traceback (most recent call last):
  File "C:\Users\Public\Anaconda3\Scripts\unitconvert-script.py", line 33, in <module>
    sys.exit(load_entry_point('unitconverter==0.1.0', 'console_scripts', 'unitconvert')())
  File "C:\Users\Public\Anaconda3\lib\site-packages\unitconverter-0.1.0-py3.7.egg\unitconverter\CLI.py", line 113, in run_cli
    if props.which == 'list':
AttributeError: 'Namespace' object has no attribute 'which'


In [7]:
'''
在 package 名稱之後，
1. 馬上要加上一個 argument，這個 TABLE 是一定要的，
   這個 TABLE 是代稱，其實有好幾個選擇，然後，
2. 要限定第二個參數，也就是  {list,convert}  operation to be performed 
   存入 which，否則 which 沒指定，就沒辦法往下作
3. 再根據 which 決定如何往下指定進一步的參數
'''
!unitconvert length

Traceback (most recent call last):
  File "C:\Users\Public\Anaconda3\Scripts\unitconvert-script.py", line 33, in <module>
    sys.exit(load_entry_point('unitconverter==0.1.0', 'console_scripts', 'unitconvert')())
  File "C:\Users\Public\Anaconda3\lib\site-packages\unitconverter-0.1.0-py3.7.egg\unitconverter\CLI.py", line 113, in run_cli
    if props.which == 'list':
AttributeError: 'Namespace' object has no attribute 'which'


In [8]:
!unitconvert length list

Unit table length can convert between the units:
mil
yd
in
ft
m
um
mm
mi
cm


In [9]:
!unitconvert length list -h

usage: unitconvert TABLE list [-h] [-m]

optional arguments:
  -h, --help    show this help message and exit
  -m, --method  Also output the conversion method from the base unit


In [10]:
'''
以 -開頭或是以 --開頭的即為 optional argument
'''
!unitconvert length list -m

Unit table length can convert between the units:
cm (x * 100.0)
um (x * 1000000.0)
mi (x * 0.00062137)
in (x * 39.370)
m (base unit)
mil (x * 39370.078740158)
yd (x * 1.0936)
ft (x * 3.2808)
mm (x * 1000.0)


In [11]:
'''
以 -開頭或是以 --開頭的即為 optional argument
'''
!unitconvert energy list -m

Unit table energy can convert between the units:
ev (x * 6241506480000000000.0)
hph (x * 3.7250614123e-7)
j (base unit)
kcal (x * 0.00023884589663)
wh (x * 0.00027777777778)
cal (x * 0.23900573614)
btu (x * 0.00094781707775)


In [12]:
'''
我們來試第二個 positional argument: convert
'''
!unitconvert time convert

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]
unitconvert TABLE convert: error: the following arguments are required: VALUE, FROM, TO


In [13]:
!unitconvert time convert -h

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


In [14]:
'''
我們再來試不同的 TABLE
'''
! unitconvert energy 

Traceback (most recent call last):
  File "C:\Users\Public\Anaconda3\Scripts\unitconvert-script.py", line 33, in <module>
    sys.exit(load_entry_point('unitconverter==0.1.0', 'console_scripts', 'unitconvert')())
  File "C:\Users\Public\Anaconda3\lib\site-packages\unitconverter-0.1.0-py3.7.egg\unitconverter\CLI.py", line 113, in run_cli
    if props.which == 'list':
AttributeError: 'Namespace' object has no attribute 'which'


In [15]:
! unitconvert energy list

Unit table energy can convert between the units:
j
hph
kcal
cal
wh
ev
btu


In [16]:
!unitconvert energy convert -h

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


In [17]:
!unitconvert energy list

Unit table energy can convert between the units:
btu
j
cal
wh
ev
hph
kcal


In [18]:
'''
試：unitconvert TABLE convert: error: the following arguments are required: VALUE, FROM, TO
'''
!unitconvert energy convert 1 wh j ev btu

1.000000 wh = 3600.000000 j
1.000000 wh = 22469423327820245893120.000000 ev
1.000000 wh = 3.412141 btu


In [19]:
! unitconvert energy list -h

usage: unitconvert TABLE list [-h] [-m]

optional arguments:
  -h, --help    show this help message and exit
  -m, --method  Also output the conversion method from the base unit


In [20]:
'''
每個單位的 joule 要轉換成其他單位的算法
'''
! unitconvert energy list -m

Unit table energy can convert between the units:
kcal (x * 0.00023884589663)
hph (x * 3.7250614123e-7)
wh (x * 0.00027777777778)
cal (x * 0.23900573614)
btu (x * 0.00094781707775)
ev (x * 6241506480000000000.0)
j (base unit)


In [21]:
! unitconvert -h energy convert

usage: unitconvert [-h] TABLE {list,convert} ...

Tool for converting units

positional arguments:
  TABLE           Unit table to use in conversion
  {list,convert}  operation to be performed

optional arguments:
  -h, --help      show this help message and exit


In [22]:
! unitconvert energy convert -h

usage: unitconvert TABLE convert [-h] VALUE FROM TO [TO ...]

positional arguments:
  VALUE       The value to convert
  FROM        Unit to convert from
  TO          Unit(s) to convert to

optional arguments:
  -h, --help  show this help message and exit


In [23]:
'''
以下是把 多少單位的 from  轉換成多少單位的 to
'''
! unitconvert energy convert 1000 j wh kcal

1000.000000 j = 0.277778 wh
1000.000000 j = 0.238846 kcal


## python argparse 的介紹 (python 的引數)
https://medium.com/@sean22492249/python-argpase-的介紹-python-的引數-26d54db52b1f